# Parser at a glance

## Construindo uma parser tree

Considere a seguinte gramatica. Vamos desenvolver um parser para reconhecer sentenças nesta gramatica e construir a parse tree para elas.

In [ ]:
# Grammar
'''
statements : statements statement
           | statement

statement : ID EQ expr
          | PRINT LPAREN expr RPAREN

expr : expr PLUS expr
     | expr TIMES expr
     | NUM
     | ID
     | LPAREN expr RPAREN
'''

## Exemplo de sentenças válidas para esta gramatica

## Analisador Léxico
O primeiro passo é construir um analisador léxico para os terminais desta gramatica:

In [1]:
from ply.lex import lex

# tokens
tokens = ('ID', 'NUM', 'PLUS', 'TIMES', 'EQ', 'LPAREN', 'RPAREN', 'PRINT',)

def t_ID(t):
    r'[a-zA-Z_][a-zA-Z0-9_]*'
    if t.value == 'print':
        t.type = "PRINT"
    return t

def t_NUM(t):
    r'[0-9]+'
    t.value = int(t.value)
    return t

t_PLUS = r'\+'
t_TIMES = r'\*'
t_EQ = r'='
t_LPAREN = r'\('
t_RPAREN = r'\)'

def t_error(t):
    print(f'Illegal character {t.value[0]}')
    t.lexer.skip(1)

t_ignore = ' \t'

__file__ = 'parser_at_a_glance.ipynb'    # necessário apenas no contexto do jupyter
lexer = lex()

lexer.input("a = 3 * 4 + 5")
for tok in lexer:
    print(tok)

LexToken(ID,'a',1,0)
LexToken(EQ,'=',1,2)
LexToken(NUM,3,1,4)
LexToken(TIMES,'*',1,6)
LexToken(NUM,4,1,8)
LexToken(PLUS,'+',1,10)
LexToken(NUM,5,1,12)


## 1o. passo do Parser: Reconhecendo as setenças

In [2]:
from ply.yacc import yacc


def p_statement_list(p):
    ''' statements : statements statement
                   | statement
    '''

def p_assign_statement (p):
    ''' statement : ID EQ expr
    '''
    
def p_print_statement (p):
    ''' statement : PRINT LPAREN expr RPAREN
    '''
    
def p_binop_expr (p):
    ''' expr : expr PLUS expr
             | expr TIMES expr
    '''
    
def p_num_expr (p):
    ''' expr : NUM
    '''
    
def p_name_expr (p):
    ''' expr : ID
    '''
    
def p_compound_expr (p):
    ''' expr : LPAREN expr RPAREN
    '''
    
parser = yacc(write_tables=False)

Generating LALR tables


Vamos ignorar os warnings por enquanto e tentar fazer o parser para uma sentença válida:

In [3]:
parser.parse('a = 3 * 4 + 5')

 Nada ocorreu! Vamos ver o que acontece com uma sentença não válida

In [4]:
parser.parse('a == 3')

yacc: Syntax error at line 1, token=EQ


Vamos acrescentar informações para construir a parse tree:

In [5]:
from ply.yacc import yacc


def p_statement_list(p):
    ''' statements : statements statement
                   | statement
    '''
    if len(p) == 2:
        p[0] = p[1]
    else:
        p[0] = p[1] + (p[2]) 

def p_assign_statement (p):
    ''' statement : ID EQ expr
    '''
    p[0] = ('assign', p[1], p[3])
    
def p_print_statement (p):
    ''' statement : PRINT LPAREN expr RPAREN
    '''
    p[0] = ('print', p[3])
    
def p_binop_expr (p):
    ''' expr : expr PLUS expr
             | expr TIMES expr
    '''
    p[0] = (p[2], p[1], p[3])
    
def p_num_expr (p):
    ''' expr : NUM
    '''
    p[0] = ('num', p[1])
    
def p_name_expr (p):
    ''' expr : ID
    '''
    p[0] = ('id', p[1])
    
def p_compound_expr (p):
    ''' expr : LPAREN expr RPAREN
    '''
    p[0] = p[2]
    
parser = yacc(write_tables=False)

Generating LALR tables


In [6]:
parser.parse("a = 3 * 4 + 5")

('assign', 'a', ('*', ('num', 3), ('+', ('num', 4), ('num', 5))))

Construida a parser tree notamos que ela não está respeitando a precedência dos operadores. Precisamos indicar isto no programa. Vamos aproveitar também e definir uma rotina de erro para o parser:

In [7]:
from ply.yacc import yacc


def p_statement_list(p):
    ''' statements : statements statement
                   | statement
    '''
    if len(p) == 2:
        p[0] = p[1]
    else:
        p[0] = p[1] + (p[2]) 

def p_assign_statement (p):
    ''' statement : ID EQ expr
    '''
    p[0] = ('assign', p[1], p[3])
    
def p_print_statement (p):
    ''' statement : PRINT LPAREN expr RPAREN
    '''
    p[0] = ('print', p[3])
    
def p_binop_expr (p):
    ''' expr : expr PLUS expr
             | expr TIMES expr
    '''
    p[0] = (p[2], p[1], p[3])
    
def p_num_expr (p):
    ''' expr : NUM
    '''
    p[0] = ('num', p[1])
    
def p_name_expr (p):
    ''' expr : ID
    '''
    p[0] = ('id', p[1])
    
def p_compound_expr (p):
    ''' expr : LPAREN expr RPAREN
    '''
    p[0] = p[2]
    
def p_error (p):
    if p:
        print("Error near the symbol %s" % p.value)
    else:
        print("Error at the end of input")

precedence = (
    ('left', 'PLUS'),
    ('left', 'TIMES')
    )
    
parser = yacc(write_tables=False)

Generating LALR tables


Ok. Com a precedência, os conflitos shift-reduce foram resolvidos. Vamos executar o parser para as nossas sentenças de exemplo novamente:

In [8]:
parser.parse('a = 3 * 4 + 5')

('assign', 'a', ('+', ('*', ('num', 3), ('num', 4)), ('num', 5)))

In [9]:
parser.parse('a = 3 * ')

Error at the end of input


In [10]:
parser.parse('a == 3')

Error near the symbol =


In [11]:
parser.parse('print(a)')

('print', ('id', 'a'))

In [12]:
code = '''
a = 3
b = 4 * a
print(a+b)
'''

In [13]:
parser.parse(code.__str__().replace('\n', ' '))

('assign',
 'a',
 ('num', 3),
 'assign',
 'b',
 ('*', ('num', 4), ('id', 'a')),
 'print',
 ('+', ('id', 'a'), ('id', 'b')))